In [ ]:
# from google.cloud import bigquery
# from google.cloud import storage


def save_in_storage(local_file_name, storage_file_name, local_path_to_file, storage_paht_to_save):
    """
    local_file_name: nome da tabela local
    storage_file_name: nome da tabela que sera criada no storage
    local_path_to_file: caminho para a pasta local onde se encontra a tabela
    storage_paht_to_save: nome da pasta onde os dados serão salvos no storage    
    """
    file_path_csv = "{}/{}.csv".format(local_path_to_file,local_file_name)
    file_path_parquet = "{}/{}.parquet".format(local_path_to_file,local_file_name)
    pd.read_csv(file_path_csv).to_parquet(file_path_parquet, index=False)
    storage_file_name_parquet = "{}.parquet".format(storage_file_name)
    
    client = storage.Client(project='gabinete-sv')
    bucket = client.get_bucket('gabinete-sv')
              
    blob = bucket.blob('{}/{}'.format(storage_paht_to_save,storage_file_name_parquet))
    blob.upload_from_filename(file_path_parquet)
    


def create_in_bquery(sorage_file_name,bquery_table_name,conjunto_de_dados,sorage_paht_to_file):
    """
    bquery_table_name: nome da tabela que sera criada no bigquery
    sorage_file_name: nome da tabela do storage
    conjunto de dados: nome do schema onde sera salvo a tabela
    sorage_paht_to_file: nome da pasta onde se encontra os dados no storage    
    """

    client = bigquery.Client( project='gabinete-sv')
    
    dataset_ref = client.dataset(conjunto_de_dados)
    
    #nome da tabela que será criada no big query
    table_name = dataset_ref.table(bquery_table_name)
    
    job_config = bigquery.LoadJobConfig()
    #overwite table
    job_config.write_disposition = "WRITE_TRUNCATE"
    #source format
    job_config.source_format = bigquery.SourceFormat.PARQUET
    
    
    uri = "gs://gabinete-sv/{}/{}.parquet".format(sorage_paht_to_file,sorage_file_name)
        
    
    load_job = client.load_table_from_uri(uri, table_name, job_config=job_config)  # API request
    
    print("Starting job {}".format(load_job.job_id))

    load_job.result()  # Waits for table load to complete.
    print("Job finished.")

    destination_table = client.get_table(dataset_ref.table(bquery_table_name))
    print("Loaded {} rows.".format(destination_table.num_rows))
    
    

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import xmltodict

import untangle

import numpy as np
import requests
import pandas as pd

pd.options.display.max_rows = 9999
pd.set_option('display.width', 1000)




import warnings
warnings.filterwarnings('ignore')

from gcloud import storage
from oauth2client.service_account import ServiceAccountCredentials
import os

import pandas as pd
import geopandas as gpd
pd.options.display.max_columns = 999

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, offline
offline.init_notebook_mode(connected=True)
import plotly.express as px

from mpl_toolkits.axes_grid1 import make_axes_locatable
%pylab inline
pylab.rcParams['figure.figsize'] = (12, 12)
import seaborn as sns



# For changes in .py
%reload_ext autoreload
%autoreload 2

import numpy as np

import missingno as msno

stats = ['skew', 'mad', 'kurt']


import qgrid

def qg(df):
    return(qgrid.show_grid(df,show_toolbar=True, grid_options={'forceFitColumns': False}))

import itertools
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)
pd.set_option('display.width', -1)
pd.set_option('display.max_colwidth', 100)

import requests
import json
import xmltodict
import re
import urllib.request as urllib2
import io
import yaml

import math
import pysal as ps

# from pysal.esda.mapclassify import Quantiles, Equal_Interval, Fisher_Jenks



import imageio


from sklearn import preprocessing


from google.cloud import bigquery
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/mnt/AEB0CCA7B0CC777D/jlab/gabinete_sv_credentials.json"

from io import BytesIO
from zipfile import ZipFile


from datetime import date
today = str(date.today()).replace('-','_')


from os import listdir
from os.path import isfile, join
import os.path
from os import path

import wget

import gc

Populating the interactive namespace from numpy and matplotlib


In [2]:
import rpy2.robjects.packages as rpackages
from rpy2.robjects import pandas2ri
utils = rpackages.importr('read.dbc')

In [178]:
anos   = ['18','19']
months = ['01','02','03','04','05','06','07','08','09','10','11','12']


for ano in anos:
    for mes in months:
        file = 'SPSP{}{}.dbc'.format(ano,mes)
        url =     'ftp://ftp.datasus.gov.br/dissemin/publicos/SIHSUS/200801_/Dados/{}'.format(file)
        try:
            wget.download(url, out='SIHSUS/dbc')
            print(file)
        except:
            print('Error on file: {}'.format(file))

In [3]:
path_dbc = 'SIHSUS/dbc'
path_parquet = 'SIHSUS/parquet'
onlyfiles = [f for f in listdir(path_dbc) if isfile(join(path_dbc, f))]
file_dbc = ["{}/{}".format(path_dbc,file) for file in onlyfiles]
file_parquet = ["{}/{}".format(path_parquet,file.replace('.dbc','.parquet')) for file in onlyfiles]

In [ ]:
for dbc_file, parquet_file in zip(file_dbc, file_parquet):
    
    if path.exists(parquet_file):
        print(parquet_file)
        pass
    else:
        r_df = utils.read_dbc(dbc_file)
        pd.DataFrame.from_dict({ key : np.asarray(r_df.rx2(key)) for key in r_df.names }).to_parquet(parquet_file, index=False)
        del r_df
        gc.collect()
        print(parquet_file)

SIHSUS/parquet/SPSP1801.parquet
SIHSUS/parquet/SPSP1802.parquet
SIHSUS/parquet/SPSP1803.parquet
SIHSUS/parquet/SPSP1804.parquet
SIHSUS/parquet/SPSP1805.parquet
SIHSUS/parquet/SPSP1806.parquet
SIHSUS/parquet/SPSP1807.parquet
SIHSUS/parquet/SPSP1808.parquet
SIHSUS/parquet/SPSP1809.parquet
SIHSUS/parquet/SPSP1810.parquet
SIHSUS/parquet/SPSP1811.parquet
SIHSUS/parquet/SPSP1812.parquet
